This lab gives an overview of the Nvidia Nsight Tool and steps to profile an application with Nsight Systems command line interface with NVTX API. You will learn how to integrate NVTX markers in your application to trace CPU events when profiling using Nsight tools. 

Let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

# NVIDIA Profiler

### What is profiling
Profiling is the first step in optimizing and tuning your application. Profiling an application would help us understand where most of the execution time is spent. You will gain an understanding of its performance characteristics and can easily identify parts of the code that present opportunities for improvement. Finding hotspots and bottlenecks in your application, can help you decide where to focus our optimization efforts.

### NVIDIA Nsight Tools
NVIDIA offers Nsight tools (Nsight Systems, Nsight Compute, Nsight Graphics), a collection of applications which enable developers to debug, profile the performance of CUDA, OpenACC, or OpenMP applications. 

Your profiling workflow will change to reflect the individual Nsight tools. Start with Nsight Systems to get a system-level overview of the workload and eliminate any system level bottlenecks, such as unnecessary thread synchronization or data movement, and improve the system level parallelism of your algorithms. Once you have done that, then proceed to Nsight Compute or Nsight Graphics to optimize the most significant CUDA kernels or graphics workloads, respectively. Periodically return to Nsight Systems to ensure that you remain focused on the largest bottleneck. Otherwise the bottleneck may have shifted and your kernel level optimizations may not achieve as high of an improvement as expected.

- **Nsight Systems** analyze application algorithm system-wide
- **Nsight Compute** debug and optimize CUDA kernels 
- **Nsight Graphics** debug and optimize graphic workloads

<img src="images/Nsight Diagram.png" width="80%" height="80%">
*The data flows between the NVIDIA Nsight tools.*

In this lab, we only focus on Nsight Systems to get the system-wide actionable insights to eliminate bottlenecks.

### Introduction to Nsight Systems 
Nsight Systems tool offers system-wide performance analysis in order to visualize application’s algorithms, help identify optimization opportunities, and improve the performance of applications running on a system consisting of multiple CPUs and GPUs.

#### Nsight Systems Timeline
- CPU rows help locating CPU core's idle times. Each row shows how the process' threads utilize the CPU cores.
<img src="images/cpu.png" width="80%" height="80%">

- Thread rows shows a detailed view of each thread's activity including OS runtime libraries usage, CUDA API calls, NVTX time ranges and events (if integrated in the application).
<img src="images/thread.png" width="80%" height="80%">

- CUDA Workloads rows display Kernel and memory transfer activites. 
<img src="images/cuda.png" width="80%" height="80%">

### Profiling using command line interface 
To profile your application, you can either use the Graphical User Interface(GUI) or Command Line Interface (CLI). During this lab, we will profile the mini application using CLI.

The Nsight Systems command line interface is named `nsys`. Below is a typical command line invocation:

`nsys profile -t openacc,nvtx --stats=true --force-overwrite true -o laplace ./laplace`

where command switch options used for this lab are:
- `profile` – start a profiling session
- `-t`: Selects the APIs to be traced (nvtx and openacc in this example)
- `--stats`: if true, it generates summary of statistics after the collection
- `--force-overwrite`e: if true, it overwrites the existing generated report
- `-o` – name for the intermediate result file, created at the end of the collection (.qdrep filename)

**Note**: You do not need to memorize the profiler options. You can always run `nsys --help` or `nsys [specific command] --help` from the command line and use the necessary options or profiler arguments.
For more info on Nsight profiler and NVTX, please see the __[Profiler documentation](https://docs.nvidia.com/nsight-systems/)__.

### How to view the report
<a name="gui-report"></a>
When using CLI to profile the application, there are two ways to view the profiler's report. 

1) On the Terminal using `--stats` option: By using `--stats` switch option, profiling results are displayed on the console terminal after the profiling data is collected.

<img src="images/laplas3.png" width="100%" height="100%">

2) NVIDIA Nsight System GUI: After the profiling session ends, a `*.qdrep` file will be created. This file can be loaded into Nsight Systems GUI using *File -> Open*. If you would like to view this on your local machine, this requires that the local system has CUDA toolkit installed of same version and the Nsight System GUI version should match the CLI version. More details on where to download CUDA toolkit can be found in the “Links and Resources” at the end of this page.

To view the profiler report, simply open the file from the GUI (File > Open).

<img src="images/nsight_open.png" width="80%" height="80%">

# Using NVIDIA Tools Extension (NVTX) 
<a name="nvtx"></a>
NVIDIA Tools Extension (NVTX) is a C-based Application Programming Interface (API) for annotating events, time ranges and resources in applications. NVTX brings the profiled application’s logic into the Profiler, making the Profiler’s displayed data easier to analyse and enables correlating the displayed data to profiled application’s actions.  

During this lab, we profile the application using Nsight Systems command line interface and collect the timeline. We will also be tracing NVTX APIs (already integrated into the application). The NVTX tool is a powerful mechanism that allows users to manually instrument their application. NVIDIA Nsight Systems can then collect the information and present it on the timeline. It is particularly useful for tracing of CPU events and time ranges and greatly improves the timeline's readability. 

**How to use NVTX**: Add `#include "nvtx3/nvToolsExt.h"` in your source code and wrap parts of your code which you want to capture events with calls to the NVTX API functions. For example, try adding `nvtxRangePush("main")` in the beginning of your `main()` function, and `nvtxRangePop(`) just before the return statement in the end.

The sample code snippet below shows the use of range events.The resulting NVTX markers can be viewed in Nsight Systems timeline view. 

```cpp
    nvtxRangePushA("init");
    initialize(A, Anew, m, n);
    nvtxRangePop();

    printf("Jacobi relaxation Calculation: %d x %d mesh\n", n, m);

    double st = omp_get_wtime();
    int iter = 0;

    nvtxRangePushA("while");
    while ( error > tol && iter < iter_max )
    {
        nvtxRangePushA("calc");
        error = calcNext(A, Anew, m, n);
        nvtxRangePop();

        nvtxRangePushA("swap");
        swap(A, Anew, m, n);
        nvtxRangePop();

        if(iter % 100 == 0) printf("%5d, %0.6f\n", iter, error);

        iter++;
    }
    nvtxRangePop();
   
```

<img src="images/nvtx.PNG" width="80%" height="80%">

**Using NVTX with Fortran** Being a C API in order to use NVTX in Fortran, wrappers must be written calling C API and exposed as a module. As part of this tutorial the file `nvtx.f90` consists of wrapper sub routines for NVTX API.  

Detailed NVTX documentation can be found under the __[CUDA Profiler user guide](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvtx)__.

### Steps to follow
To obtain the best performance from GPU and utilize the hardware, one should follow the cyclical process (analyze, parallelize, optimize). 

- **Analyze**: In this step, you first identify the portion of your code that includes most of the computation and most of the execution time is spent. From here, you find the hotspots, evaluate the bottlenecks and start investigating GPU acceleration.

- **Parallelize**: Now that we have identified the bottlenecks, we use use the techniques to paralellise the routines where most of the time is spent.

- **Optimize**:  To further improve the performance, one can implement optimization strategies step by step in an iterative process including: identify optimization opportunity, apply and test the optimization method, verify and repeat the process.

Note: The above optimization is done incrementally after investigating the profiler output.

We will follow the optimization cycle for porting and improving the code performance.

<img src="images/Optimization_Cycle.jpg" width="80%" height="80%">

# Getting Started 
In the following sections, we parallelise and optimize the serial [RDF](../../../nways_MD/English/C/jupyter_notebook/serial/rdf_overview.ipynb) using different approaches to the GPU programming following the above steps. For each section, inspect the code, compile, and profile it. Then, investigate the profiler’s report to identify the bottlenecks and spot the optimization opportunities.  At each step, locate problem areas in the application and make improvements iteratively to increase performance.

This lab comprises of multiple exercises, each follows the optimization cycle method. For each exercise, compile the code, validate the output (more instruction in the labs) and profile it. You will profile the code with Nsight Systems (`nsys`), identify certain areas/kernels in the code, where they don't behave as expected. 

**NOTE**: Example screenshots are for reference only and you may not get identical profiler report.

-----

# <div style="text-align: center ;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em">[HOME](../../../nways_start.ipynb)</div>

-----

# Links and Resources


[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)


**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System's latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0). 